# Wyjaśnienie Metod z `exercise2.py`

Ten notatnik wyjaśnia koncepcje i techniki użyte w skrypcie `exercise2.py`. Obejmuje on dwa główne ćwiczenia:
1.  Obliczanie entropii hasła.
2.  Atak brute-force na plik zaszyfrowany algorytmem AES w trybie ECB.

## Ćwiczenie 1: Entropia Hasła

### Czym jest Entropia?

Entropia w kontekście bezpieczeństwa informatycznego to miara nieprzewidywalności lub losowości informacji. W przypadku haseł, entropia mierzy, jak trudne jest odgadnięcie hasła. Mierzy się ją w bitach.

-   **Entropia 0 bitów**: Hasło jest znane (np. "password"). Nie ma żadnej niepewności.
-   **Entropia N bitów**: Istnieje 2<sup>N</sup> możliwych kombinacji hasła. Atakujący musiałby średnio sprawdzić 2<sup>N-1</sup> możliwości, aby je odgadnąć.

### Jak Obliczyć Entropię?

Wzór na entropię hasła jest następujący:

**E = L * log<sub>2</sub>(N)**

Gdzie:
-   **E** - Entropia całkowita (w bitach).
-   **L** - Długość hasła (liczba znaków).
-   **N** - Rozmiar zestawu znaków, z których może składać się hasło (np. 26 dla `a-z`, 52 dla `a-zA-Z`, 62 dla `a-zA-Z0-9`).

### Implementacja w `exercise2.py`

W skrypcie `solve_exercise_1` obliczamy, jak długie musi być hasło, aby osiągnąć entropię 256 bitów, co jest standardem dla bezpiecznego klucza AES-256.

1.  **Definiujemy rozmiar alfabetu (N)**: Dla małych liter `a-z`, `N = 26`.
2.  **Obliczamy entropię na znak**: `log2(26) ≈ 4.70 bita`. Oznacza to, że każdy losowy znak z tego zestawu dodaje 4.70 bita niepewności.
3.  **Obliczamy wymaganą długość (L)**: Aby osiągnąć 256 bitów, dzielimy entropię docelową przez entropię na znak:
    `L = 256 / 4.70 ≈ 54.46`

Ponieważ długość hasła musi być liczbą całkowitą, zaokrąglamy wynik w górę.

**Wniosek**: Potrzeba co najmniej **55 losowo wybranych małych liter**, aby stworzyć hasło o sile porównywalnej z kluczem AES-256.

## Ćwiczenie 2: Atak Brute-Force na AES w trybie ECB

### Kontekst Ataku

W tym ćwiczeniu mamy plik `security_ECB_encrypted.bmp`, który został zaszyfrowany algorytmem **AES** w trybie **ECB (Electronic Codebook)**. Naszym celem jest odzyskanie oryginalnej zawartości pliku.

Otrzymaliśmy kluczową wskazówkę:
> Hasło to 16-krotnie powtórzony znak z zestawu `a-z` lub `0-9`.

To drastycznie zmniejsza przestrzeń poszukiwań i sprawia, że atak brute-force jest wykonalny.

### AES i Tryb ECB

-   **AES (Advanced Encryption Standard)**: Symetryczny szyfr blokowy, uważany za bardzo bezpieczny. Działa na blokach danych o stałym rozmiarze (128 bitów).
-   **Tryb ECB (Electronic Codebook)**: Najprostszy tryb działania szyfrów blokowych. Każdy blok danych jest szyfrowany niezależnie przy użyciu tego samego klucza.
    -   **Wada ECB**: Identyczne bloki tekstu jawnego po zaszyfrowaniu dają identyczne bloki szyfrogramu. To pozwala na rozpoznanie wzorców w zaszyfrowanych danych, co jest dużą słabością, szczególnie w przypadku obrazów.

### Logika Ataku Brute-Force

Atak polega na systematycznym sprawdzaniu każdego możliwego klucza, aż do znalezienia właściwego.

1.  **Generowanie Klucza**:
    -   Iterujemy po wszystkich możliwych znakach (`a-z`, `0-9`).
    -   Dla każdego znaku `c`, tworzymy hasło `c` powtórzone 16 razy (np. `"aaaaaaaaaaaaaaaa"`).
    -   Wskazówka sugeruje, że to 16-bajtowe hasło jest **bezpośrednio kluczem AES-128**. Nie używamy funkcji skracającej (hashującej) jak SHA-256, ponieważ klucz ma już prawidłową długość (16 bajtów = 128 bitów).

2.  **Deszyfrowanie**:
    -   Dla każdego wygenerowanego klucza próbujemy odszyfrować zawartość pliku `security_ECB_encrypted.bmp`.
    -   Używamy `AES.new(key, AES.MODE_ECB)`.

3.  **Walidacja (Sprawdzenie Poprawności Klucza)**:
    Skąd wiemy, że znaleźliśmy właściwy klucz? Musimy mieć sposób na weryfikację odszyfrowanych danych.
    -   **Padding (Dopełnienie)**: AES działa na pełnych blokach. Jeśli ostatni blok danych jest niepełny, dodaje się do niego dopełnienie (padding) w standardzie PKCS7. Jeśli klucz jest nieprawidłowy, próba usunięcia dopełnienia (`unpad`) najprawdopodobniej zakończy się błędem `ValueError`. To nasz pierwszy filtr.
    -   **Sygnatura Pliku (Magic Number)**: Jeśli usunięcie dopełnienia się powiedzie, musimy jeszcze sprawdzić, czy odszyfrowane dane mają sens. Wiemy, że oryginalny plik to obraz `.bmp`. Pliki BMP mają stałą sygnaturę - zawsze zaczynają się od dwóch bajtów: `0x42 0x4D`, co w ASCII odpowiada literom **"BM"**. Sprawdzamy, czy odszyfrowane dane zaczynają się od `b'BM'`.

4.  **Sukces**:
    -   Jeśli oba warunki (poprawne usunięcie dopełnienia i sygnatura 'BM') są spełnione, oznacza to, że znaleźliśmy właściwy klucz.
    -   Zapisujemy odszyfrowane dane do nowego pliku `security_decrypted.bmp`.

### Wyniki Ataku - Sukces!

Po zaimplementowaniu opisanej logiki, skrypt **pomyślnie znalazł klucz**: literę `'p'` powtórzoną 16 razy (`pppppppppppppppp`). Atak zakończył się sukcesem po sprawdzeniu 16 możliwych kluczy (od 'a' do 'p').

#### Szczegóły Sukcesu:
- **Znaleziony klucz**: `pppppppppppppppp`
- **Rozmiar odszyfrowanego pliku**: 123,824 bajty
- **Wymiary obrazu**: 448×274 piksele
- **Głębia kolorów**: 8-bit (256 kolorów)
- **Poprawna sygnatura BMP**: Plik zaczyna się od `BM`

#### Jak Działa Szyfrowanie ECB?

**AES w trybie ECB** dzieli dane na bloki 16-bajtowe i szyfruje każdy blok niezależnie:

```
Blok 1: [16 bajtów danych] → AES(klucz) → [16 bajtów szyfrogramu]
Blok 2: [16 bajtów danych] → AES(klucz) → [16 bajtów szyfrogramu]
...
```

**Główne cechy ECB:**
1. **Determinizm**: Identyczne bloki danych dają identyczne bloki szyfrogramu
2. **Brak propagacji**: Błąd w jednym bloku nie wpływa na inne
3. **Równoległość**: Każdy blok można szyfrować/deszyfrować niezależnie

#### Jak Sprawdzamy Poprawność Deszyfrowania?

Mamy kilka metod weryfikacji:

1. **Sygnatura pliku (Magic Number)**:
   - Pliki BMP zawsze zaczynają się od `0x42 0x4D` ("BM" w ASCII)
   - To najpewniejszy sposób weryfikacji dla plików BMP

2. **Struktura nagłówka BMP**:
   - Bajty 2-5: Rozmiar pliku (little-endian)
   - Bajty 18-21: Szerokość obrazu
   - Bajty 22-25: Wysokość obrazu
   - Wartości te powinny być rozsądne

3. **Entropia danych**:
   - Poprawnie odszyfrowane dane obrazu mają charakterystyczną entropię
   - Błędnie odszyfrowane dane wyglądają jak losowy szum

4. **Padding PKCS7**:
   - Jeśli użyto paddingu, jego usunięcie musi się powieść
   - W naszym przypadku plik nie wymagał paddingu

#### Problem z Rozmiarem Pliku

Oryginalny plik miał 123,830 bajtów, ale AES wymaga wielokrotności 16 bajtów:
- `123,830 % 16 = 6` (pozostałość 6 bajtów)
- Musieliśmy obciąć do 123,824 bajtów dla kompatybilności z AES
- Te 6 bajtów to prawdopodobnie metadane systemu plików, a nie część obrazu

## Bardziej Zaawansowane Metody Ataku (Bez Wskazówek)

### 1. Atak na Klucz Pełnej Długości

**Problem**: Bez wskazówki przestrzeń kluczy AES-128 to 2^128 możliwości - praktycznie niemożliwe do przełamania brute-force.

**Rozwiązanie**: Szukanie słabości w generowaniu kluczy:
- Analiza entropii klucza
- Sprawdzanie popularnych haseł/wzorców
- Atak słownikowy z transformacjami

### 2. Analiza Wzorców ECB

**Słabość ECB**: Identyczne bloki dają identyczne szyfrogramy.

```python
def analyze_ecb_patterns(ciphertext):
    """Analizuje powtarzające się bloki w ECB"""
    blocks = [ciphertext[i:i+16] for i in range(0, len(ciphertext), 16)]
    block_counts = {}
    
    for block in blocks:
        if block in block_counts:
            block_counts[block] += 1
        else:
            block_counts[block] = 1
    
    # Bloki powtarzające się więcej niż raz
    repeated = {k: v for k, v in block_counts.items() if v > 1}
    return repeated, len(set(blocks)) / len(blocks)  # Unikalność bloków
```

### 3. Atak Known-Plaintext

Jeśli znamy fragment oryginalnych danych:

```python
def known_plaintext_attack(known_plain, known_cipher, full_cipher):
    """Próba znalezienia klucza przy znanym fragmencie"""
    # Dla ECB: jeśli znamy plaintext i ciphertext bloku,
    # możemy sprawdzić wszystkie możliwe klucze
    for candidate_key in generate_candidate_keys():
        cipher = AES.new(candidate_key, AES.MODE_ECB)
        if cipher.encrypt(known_plain) == known_cipher:
            # Znaleźliśmy klucz!
            return cipher.decrypt(full_cipher)
    return None
```

### 4. Frequency Analysis dla Obrazów

Obrazy mają charakterystyczne wzorce:

```python
def image_frequency_analysis(decrypted_data):
    """Analiza częstotliwości dla weryfikacji obrazów"""
    if len(decrypted_data) < 54:  # Minimalny nagłówek BMP
        return False
    
    # Sprawdź sygnaturę
    if not decrypted_data.startswith(b'BM'):
        return False
    
    # Sprawdź rozsądność wymiarów
    try:
        width = int.from_bytes(decrypted_data[18:22], 'little')
        height = int.from_bytes(decrypted_data[22:26], 'little')
        
        if width <= 0 or height <= 0 or width > 10000 or height > 10000:
            return False
            
        # Sprawdź spójność rozmiaru pliku
        expected_size = 54 + width * height  # Uproszczone dla 8-bit
        actual_size = len(decrypted_data)
        
        return abs(expected_size - actual_size) < width  # Tolerancja
    except:
        return False
```

### 5. Atak Hybrydowy

Kombinacja różnych technik:

```python
def hybrid_attack(ciphertext):
    """Atak łączący różne metody"""
    
    # 1. Sprawdź wzorce ECB
    patterns, uniqueness = analyze_ecb_patterns(ciphertext)
    
    if uniqueness < 0.95:  # Dużo powtórzeń
        print("Wykryto wzorce ECB - możliwy słaby klucz")
    
    # 2. Słownik popularnych kluczy
    common_patterns = [
        lambda c: c * 16,  # Powtórzone znaki
        lambda c: (c + '0123456789012345')[:16],  # Znak + cyfry
        lambda c: hashlib.md5(c.encode()).digest(),  # Hash z znaku
    ]
    
    # 3. Próbuj różne transformacje
    for char in 'abcdefghijklmnopqrstuvwxyz0123456789':
        for pattern_func in common_patterns:
            key = pattern_func(char)
            if test_key(key, ciphertext):
                return key
                
    return None
```

### 6. Side-Channel Attacks

W realnym scenariuszu:
- **Timing attacks**: Analiza czasu operacji
- **Power analysis**: Analiza zużycia energii
- **Cache attacks**: Analiza dostępu do pamięci cache

### 7. Differential Cryptanalysis

Analiza różnic między szyfrogramami:

```python
def differential_analysis(file1_path, file2_path):
    """Analiza różnicowa dwóch podobnych plików zaszyfrowanych tym samym kluczem"""
    with open(file1_path, 'rb') as f1, open(file2_path, 'rb') as f2:
        data1, data2 = f1.read(), f2.read()
    
    # Znajdź bloki, które się różnią
    different_blocks = []
    for i in range(0, min(len(data1), len(data2)), 16):
        block1 = data1[i:i+16]
        block2 = data2[i:i+16]
        if block1 != block2:
            different_blocks.append((i//16, block1, block2))
    
    return different_blocks
```

### 8. Template Attacks

Wykorzystanie znanej struktury plików:

```python
def template_attack_bmp(ciphertext):
    """Atak używający szablonu struktury BMP"""
    # Znamy, że pierwszy blok zawiera nagłówek BMP
    first_block = ciphertext[:16]
    
    # Próbujemy różne klucze i sprawdzamy, czy pierwszy blok
    # po deszyfrowaniu zaczyna się od "BM"
    for candidate_key in generate_weak_keys():
        cipher = AES.new(candidate_key, AES.MODE_ECB)
        decrypted_block = cipher.decrypt(first_block)
        
        if decrypted_block.startswith(b'BM'):
            # Sprawdź pełny plik
            full_decrypted = cipher.decrypt(ciphertext)
            if validate_bmp_structure(full_decrypted):
                return candidate_key, full_decrypted
    
    return None, None
```

## Wnioski i Najlepsze Praktyki

### Słabości Wykryte w Tym Ćwiczeniu:

1. **Słaby klucz**: Powtarzanie pojedynczego znaku 16 razy drastycznie zmniejsza entropię
2. **Tryb ECB**: Determinizm i brak propagacji błędów
3. **Brak dodatkowych zabezpieczeń**: Brak HMAC czy innych mechanizmów integralności

### Zalecenia Bezpieczeństwa:

1. **Użyj CBC zamiast ECB**: Tryb CBC wprowadza wzajemne zależności między blokami
2. **Generuj klucze z wysoką entropią**: Używaj kryptograficznie bezpiecznych generatorów liczb losowych
3. **Dodaj weryfikację integralności**: HMAC lub authenticated encryption (AES-GCM)
4. **Użyj bibliotek kryptograficznych**: Nie implementuj kryptografii samodzielnie
5. **Zastosuj key stretching**: PBKDF2, bcrypt, lub Argon2 dla kluczy pochodzących z haseł

### Dlaczego Ten Atak Był Możliwy:

- **Ograniczona przestrzeń kluczy**: Tylko 36 możliwości (26 liter + 10 cyfr)
- **Znana struktura pliku**: BMP ma przewidywalny nagłówek
- **Deterministyczny ECB**: Identyczne bloki → identyczne szyfrogramy
- **Brak dodatkowych warstw bezpieczeństwa**: Tylko surowe szyfrowanie AES-ECB

Ten przykład doskonale ilustruje, jak pojedyncza słabość (słaby klucz) może skompromitować cały system, niezależnie od tego, jak silny jest użyty algoritm szyfrowania (AES).

## Automatyczna Analiza Wzorców ECB w `solve_exercise_2()`

### Zintegrowana Metoda Detekcji

Funkcja `solve_exercise_2()` została rozszerzona o **automatyczną analizę wzorców ECB**, która działa **bez żadnych dodatkowych informacji** - wymaga tylko zaszyfrowanego pliku.

### Jak Działa Automatyczna Detekcja?

Przed rozpoczęciem ataku brute-force, skrypt:

1. **Dzieli plik na bloki 16-bajtowe** (rozmiar bloku AES)
2. **Liczy częstotliwość występowania** każdego unikalnego bloku
3. **Oblicza współczynnik unikalności** (liczba unikalnych bloków / liczba wszystkich bloków)
4. **Wykrywa tryb ECB** jeśli współczynnik < 0.90 (90%)

### Przykładowe Wyniki Analizy

```
[Pre-Attack Analysis] Running ECB pattern detection...
  Total blocks: 7739
  Unique blocks: 2996
  Uniqueness ratio: 0.3871 (38.71%)
  [!] ECB MODE DETECTED! Low uniqueness suggests ECB encryption.
  [!] Found 39 repeated block patterns
  [!] Most repeated block appears 3761 times (48.6% of file)
  --> This confirms ECB mode weakness - proceeding with attack...
```

### Co Nam To Mówi?

- **38.71% unikalności** = 61.29% bloków to powtórzenia!
- **Jeden blok powtarza się 3,761 razy** = zajmuje prawie połowę pliku
- To sugeruje duże obszary jednolitego koloru (tło obrazu)

### Dlaczego To Działa Bez Dodatkowych Informacji?

**Słabość ECB jest strukturalna:**
- ECB szyfruje każdy blok niezależnie
- Identyczne bloki danych → identyczne bloki szyfrogramu
- Losowe dane miałyby ~100% unikalność
- Niska unikalność = ECB + strukturalne dane

### Zastosowania Praktyczne

Ta metoda może:
1. ✅ **Wykryć tryb ECB** automatycznie
2. ✅ **Ocenić podatność** na ataki
3. ✅ **Potwierdzić strukturalne dane** (obrazy, dokumenty)
4. ✅ **Działać bez żadnych wskazówek** o kluczu czy algorytmie
5. ✅ **Zidentyfikować kandydatów** do dalszej analizy

### Porównanie z Innymi Trybami

| Tryb | Oczekiwana Unikalność | Opis |
|------|----------------------|------|
| ECB (obraz) | 30-60% | Wzorce widoczne, powtórzenia |
| CBC | ~99.9% | Wysoka losowość |
| CTR | ~99.9% | Wysoka losowość |
| GCM | ~99.9% | Wysoka losowość |

**Wniosek**: Niska unikalność jednoznacznie wskazuje na ECB i stanowi sygnał alarmowy!